In [ ]:
input_image = '/home/eferreira/master/custom_vit/datasets/dfdc/val/faces/4000_4_1.png'  # Substitua pelo caminho da sua imagem

In [5]:
import os
from PIL import Image
import numpy as np
import cv2

# Caminho da imagem de entrada
input_image = '/home/eferreira/master/custom_vit/datasets/dfdc/val/faces/4000_4_1.png'

# Verifica se o arquivo existe
if not os.path.isfile(input_image):
    print(f"Imagem não encontrada no caminho especificado: {input_image}")
    exit()

# Carrega a imagem de entrada
img = Image.open(input_image)

# Separa os canais R, G e B
r, g, b = img.split()

# Cria diretórios para salvar as imagens se não existirem
os.makedirs('results/canais_originais', exist_ok=True)
os.makedirs('results/canais_filtrados', exist_ok=True)
os.makedirs('results/filtros_individuais', exist_ok=True)

# Salva cada canal original mantendo a cor do canal
r_colored = Image.merge('RGB', (r, Image.new('L', r.size, 0), Image.new('L', r.size, 0)))
r_colored.save('results/canais_originais/canal_r.png')

g_colored = Image.merge('RGB', (Image.new('L', g.size, 0), g, Image.new('L', g.size, 0)))
g_colored.save('results/canais_originais/canal_g.png')

b_colored = Image.merge('RGB', (Image.new('L', b.size, 0), Image.new('L', b.size, 0), b))
b_colored.save('results/canais_originais/canal_b.png')

# Converte os canais para arrays NumPy
r_array = np.array(r)
g_array = np.array(g)
b_array = np.array(b)

# Define os filtros (kernels)
kernel1 = np.array([[0, 0, 0, 0, 0],
                    [0, -1, 2, -1, 0],
                    [0, 2, -4, 2, 0],
                    [0, -1, 2, -1, 0],
                    [0, 0, 0, 0, 0]], dtype=float) / 4.0

kernel2 = np.array([[-1, 2, -2, 2, -1],
                    [2, -6, 8, -6, 2],
                    [-2, 8, -12, 8, -2],
                    [2, -6, 8, -6, 2],
                    [-1, 2, -2, 2, -1]], dtype=float) / 12.0

kernel3 = np.array([[0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 1, -2, 1, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]], dtype=float) / 2.0

kernels = {
    'Filtro1': kernel1,
    'Filtro2': kernel2,
    'Filtro3': kernel3
}

# Função para aplicar um filtro a um canal
def apply_filter(channel_array, kernel):
    # Converte para float32
    channel_array = channel_array.astype(np.float32)
    # Aplica o filtro
    filtered = cv2.filter2D(channel_array, -1, kernel)
    # Limita os valores entre -3 e 3
    filtered = np.clip(filtered, -3, 3)
    # Normaliza para o intervalo 0-255
    filtered_normalized = (filtered - filtered.min()) / (filtered.max() - filtered.min()) * 255
    filtered_normalized = filtered_normalized.astype(np.uint8)
    return filtered_normalized

# Dicionário para armazenar os canais filtrados
filtered_channels = {'R': {}, 'G': {}, 'B': {}}

# Aplica cada filtro a cada canal e salva os results
for filtro_nome, kernel in kernels.items():
    # Canal Vermelho
    r_filtered = apply_filter(r_array, kernel)
    r_filtered_colored = Image.merge('RGB', (Image.fromarray(r_filtered), Image.new('L', r.size, 0), Image.new('L', r.size, 0)))
    r_filtered_colored.save(f'results/filtros_individuais/{filtro_nome}_canal_R.png')
    filtered_channels['R'][filtro_nome] = r_filtered

    # Canal Verde
    g_filtered = apply_filter(g_array, kernel)
    g_filtered_colored = Image.merge('RGB', (Image.new('L', g.size, 0), Image.fromarray(g_filtered), Image.new('L', g.size, 0)))
    g_filtered_colored.save(f'results/filtros_individuais/{filtro_nome}_canal_G.png')
    filtered_channels['G'][filtro_nome] = g_filtered

    # Canal Azul
    b_filtered = apply_filter(b_array, kernel)
    b_filtered_colored = Image.merge('RGB', (Image.new('L', b.size, 0), Image.new('L', b.size, 0), Image.fromarray(b_filtered)))
    b_filtered_colored.save(f'results/filtros_individuais/{filtro_nome}_canal_B.png')
    filtered_channels['B'][filtro_nome] = b_filtered

# Combina os filtros aplicados a cada canal
for canal in ['R', 'G', 'B']:
    # Inicializa com zeros
    combined = np.zeros_like(filtered_channels[canal]['Filtro1'], dtype=np.float32)
    # Soma os filtros
    for filtro_nome in kernels.keys():
        combined += filtered_channels[canal][filtro_nome].astype(np.float32)
    # Calcula a média
    combined /= len(kernels)
    # Normaliza para 0-255
    combined = (combined - combined.min()) / (combined.max() - combined.min()) * 255
    combined = combined.astype(np.uint8)
    # Atualiza o canal filtrado
    filtered_channels[canal]['Combinado'] = combined

# Salva os canais filtrados combinados mantendo a cor original
r_filtered_combined_colored = Image.merge('RGB', (Image.fromarray(filtered_channels['R']['Combinado']), Image.new('L', r.size, 0), Image.new('L', r.size, 0)))
r_filtered_combined_colored.save('results/canais_filtrados/canal_R_filtrado.png')

g_filtered_combined_colored = Image.merge('RGB', (Image.new('L', g.size, 0), Image.fromarray(filtered_channels['G']['Combinado']), Image.new('L', g.size, 0)))
g_filtered_combined_colored.save('results/canais_filtrados/canal_G_filtrado.png')

b_filtered_combined_colored = Image.merge('RGB', (Image.new('L', b.size, 0), Image.new('L', b.size, 0), Image.fromarray(filtered_channels['B']['Combinado'])))
b_filtered_combined_colored.save('results/canais_filtrados/canal_B_filtrado.png')

# Recombina os canais filtrados combinados
final_image = Image.merge('RGB', (Image.fromarray(filtered_channels['R']['Combinado']),
                                  Image.fromarray(filtered_channels['G']['Combinado']),
                                  Image.fromarray(filtered_channels['B']['Combinado'])))

# Cria diretório para a imagem final
os.makedirs('results/imagem_final', exist_ok=True)

# Salva a imagem final
final_image.save('results/imagem_final/imagem_final.png')

print("Processamento concluído. As imagens foram salvas na pasta 'results'.")


Processamento concluído. As imagens foram salvas na pasta 'results'.
